In [1]:
import ipywidgets as widgets
import sys
from pathlib import Path
import os
import importlib

module_path='preprocessing/day_intervals_preproc'
if module_path not in sys.path:
    sys.path.append(module_path)

module_path='utils'
if module_path not in sys.path:
    sys.path.append(module_path)
    
module_path='preprocessing'
if module_path not in sys.path:
    sys.path.append(module_path)

import day_intervals_cohort_v2
from day_intervals_cohort_v2 import *

import data_generation_icu

import feature_selection_icu
from feature_selection_icu import *

import warnings
warnings.filterwarnings('ignore')

# import ml_models
# from ml_models import *

# import dl_train
# from dl_train import *

# import tokenization
# from tokenization import *

# import behrt_train
# from behrt_train import *

# import evaluation
# import fairness
# import callibrate_output

## 1. DATA EXTRACTION



In [2]:
root_dir = os.path.dirname(os.path.abspath('UserInterface.ipynb'))
version_path="mimiciv/2.0"

In [3]:
cohort_output = day_intervals_cohort_v2.extract_data(root_dir)

===========MIMIC-IV v2.0============
EXTRACTING FOR: | ICU | READMISSION DUE TO J44 | 30 | 
[ READMISSION DUE TO J44 ]
Index(['subject_id', 'stay_id', 'hadm_id', 'intime', 'outtime', 'los',
       'min_valid_year', 'dod', 'Age', 'gender', 'race', 'insurance'],
      dtype='object')


100%|██████████████████████████████████████| 5217/5217 [00:15<00:00, 341.08it/s]


[ READMISSION LABELS FINISHED ]
[ COHORT SUCCESSFULLY SAVED ]
[ SUMMARY SUCCESSFULLY SAVED ]
Readmission FOR ICU DATA
# Admission Records: 7346
# Patients: 5217
# Positive cases: 1137
# Negative cases: 6209


## 2. FEATURE SELECTION
All features will be saved in **./data/features/**

In [4]:
feature_icu(cohort_output, version_path)

[EXTRACTING CHART EVENTS DATA]


32it [07:52, 14.77s/it]


# Unique Events:   412
# Admissions:   7345
Total rows 7568035
[SUCCESSFULLY SAVED CHART EVENTS DATA]


## 3. SUMMARY OF FEATURES

This step will generate summary of all features extracted so far.<br>
It will save summary files in **./data/summary/**<br>
- These files provide summary about **mean frequency** of medical codes per admission.<br>
- It also provides **total occurrence count** of each medical code.<br>
- For labs and chart events it will also provide <br>**missing %** which tells how many rows for a certain medical code has missing value.

Please use this information to further refine your cohort by selecting <br>which medical codes in each feature you want to keep and <br>which codes you would like to remove for downstream analysis tasks.

**Please run below cell to generate summary files**

In [5]:
generate_summary_icu(True)

[GENERATING FEATURE SUMMARY]
[SUCCESSFULLY SAVED FEATURE SUMMARY]


## 4. Feature Selection

based on the files generated in previous step and other infromation gathered by you,<br>
Please select which medical codes you want to include in this study.

In [11]:
# First get the path to the file. 
sum_path = "data/summary/chart_features.csv"

# Then use pandas to load up the dataframe.
feats = pd.read_csv(sum_path)

# Set up all of the medical codes you want to save
to_keep = [220045, 220179, 223761, 220210, 220277]

keeps = []
for i in feats["itemid"]:
    keeps.append(i in to_keep)

# Filter feats
feat_filt = feats[keeps]

# Save the features of interest
feat_filt.to_csv(sum_path, index = False)

In [12]:
# This will perform the feature selection based on your subsetted list of features. 
features_selection_icu(cohort_output)


[FEATURE SELECTION CHART EVENTS DATA]
Total number of rows 2556262
[SUCCESSFULLY SAVED CHART EVENTS DATA]


## 5. CLEANING OF FEATURES
We chose not to perform any additional outlier detection here, as we did not want to lower our sample size even further. 

In [13]:
thresh=0
clean_chart='No outlier detection'
impute_outlier_chart=False
thresh=98
left_thresh=0
preprocess_features_icu(cohort_output,clean_chart,impute_outlier_chart,thresh,left_thresh)

[PROCESSING CHART EVENTS DATA]
Total number of rows 2527135
[SUCCESSFULLY SAVED CHART EVENTS DATA]


## 6. Time-Series Representation

- We select the last 72 hours of the first admission as the time-series data we want to include for this study.

- the selection of bucket size is arbitrary as all data over those 72 hours will be aggregated in the final dataset.

Following data generation, the seperate getXY notebook is used to create the final datasets. 

In [14]:
predW=0
bucket = 1
include = 72
impute=False
gen=data_generation_icu.Generator(cohort_output,impute,include,bucket,predW)


[ READ COHORT ]
[ ======READING CHART EVENTS ]


1it [00:23, 23.42s/it]


[ READ ALL FEATURES ]
[ PROCESSED TIME SERIES TO EQUAL LENGTH  ]


100%|██████████████████████████████████████████| 72/72 [00:00<00:00, 159.28it/s]


bucket 1
[ PROCESSED TIME SERIES TO EQUAL TIME INTERVAL ]
72


100%|███████████████████████████████████████| 2538/2538 [00:44<00:00, 56.99it/s]


[ SUCCESSFULLY SAVED DATA DICTIONARIES ]
